# Import python dataset

In [ ]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

# Common imports
import numpy as np
import os

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

import pandas as pd
import time
import functools

In [ ]:
import numpy as np
import functools
import time

import matplotlib.pyplot as plt
import tensorflow as tf
import keras
from keras.optimizers import Adam
from tensorflow import reduce_mean
import gc
from sklearn.preprocessing import *


In [ ]:
import argparse
import os
import numpy as np
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Reshape, Flatten
#from keras.layers.merge import _Merge
from keras.layers.convolutional import Convolution2D, Conv2DTranspose
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import Adam
from keras.datasets import mnist
from keras import backend as K
from functools import partial

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

device_name = tf.test.gpu_device_name()
allow_pickle=True

In [ ]:
# detect and init the TPU
#tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()

# instantiate a distribution strategy
#tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

In [ ]:
notebookName = 'WGAN.ipynb'
k = 1

In [ ]:
tf.config.run_functions_eagerly(True)

In [ ]:
os.makedirs('./output/Figures/Individual')
os.makedirs('./output/Figures/Result')
os.makedirs('./output/Figures/Prediction')
os.makedirs('./output/ganmodels')
os.makedirs( './output/logs/WGAN/generator')
os.makedirs('./output/logs/WGAN/critic')
os.makedirs('./output/Figures/Loss')

## Load data

In [ ]:
n_features = 7

In [ ]:
data= np.loadtxt('../input/pollutant/01.txt') 
print(data.shape)

data02= np.loadtxt('../input/pollutant/02.txt') 
print(data02.shape)

data04= np.loadtxt('../input/pollutant/04.txt') 
print(data04.shape)


groups = ['PM10','PM2_5', 'PM1', 'CO2' ,'Temeprature' ,'RH']
groups = np.array(groups)
print(groups.shape)

groups2 = ['times','PM10','PM2_5', 'PM1', 'CO2' ,'Temeprature' ,'RH']
groups2 = np.array(groups2)
print(groups2)

In [ ]:
fig, ax = plt.subplots(1,1, figsize=[20,10])
ax.plot(data[:,0], data[:,1], label='PM10')
ax.plot(data[:,0], data[:,2], label='PM2.5')
ax.plot(data[:,0], data[:,3], label='PM1')
ax.plot(data[:,0], data[:,4], label='CO2')

ax.grid()
plt.ylabel("concentration (ppm)")
plt.xlabel("time (min)")
plt.title('Tube')
plt.legend()


fig, ax = plt.subplots(1,1, figsize=[10,5])
ax.plot(data[:,0], data[:,5],c='black', label='Temperature')
ax.grid()
plt.ylabel("RH (%)")
plt.xlabel("time (min)")
plt.title('Humidity')
plt.legend()


fig, ax = plt.subplots(1,1, figsize=[10,5])
ax.plot(data[:,0], data[:,6],c='black', label='RH') #what is the unit?
ax.grid()
plt.ylabel("Celsius")
plt.xlabel("time (min)")
plt.title('Tube')
plt.legend()

plt.savefig('./output/Figures/Sample_v'+str(k)+'.png')
plt.show

# Preparing Dataset

In [ ]:
codings_size = n_features

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=[-1,1])
X_train_scaled = scaler.fit_transform(data)

X_train_scaled.shape 


In [ ]:
#scaler inverse_transform
X_train = scaler.inverse_transform(((X_train_scaled)))
X_train.shape 

#fig, ax = plt.subplots(2,4, figsize=[20,10])
#for i, group in enumerate(groups):
    #a=i+1
#    ax.flatten()[i].plot(X_train[:,0], X_train[:,i+1])
#    ax.flatten()[i].set_title(group)
#plt.savefig('../output/Figures/Individual/Sample'+'_v'+str(k)+'.png')

# Train the GAN

### Preparing the training dataset

In [ ]:
n = 500

In [ ]:
def concat_timesteps(X_train_scaled, ntimes, step):
    X_train_concat = []
    for i in range(len(X_train_scaled) - ntimes*step):
        X_train_concat.append(X_train_scaled[i:i+ntimes*step:step])
    return np.array(X_train_concat)

ntimes = 100 # Consecutive times for the GAN
step = 1 # step between times (1 mins)

X_train_concat = concat_timesteps(X_train_scaled, ntimes, step)#X_train_scaled
X_train_concat = X_train_concat.reshape(X_train_concat.shape[0],ntimes,n_features,1)
print(X_train_concat.shape)
#Individual sample dataset


print(X_train_concat[n].shape)


In [ ]:
BATCH_SIZE = 64
latent_space = 100

# Create a tensorflow dataset and split it into batches
train_dataset = X_train_concat.reshape(X_train_concat.shape[0], ntimes, n_features,1).astype('float32')
train_dataset = tf.data.Dataset.from_tensor_slices(train_dataset)
train_dataset = train_dataset.shuffle(len(X_train_scaled))
train_dataset = train_dataset.batch(BATCH_SIZE)

for data in train_dataset:
    print(data.shape)
    break

In [ ]:
#wgan = keras.models.load_model('../input/wganmodels/v1_epoch_2700.h5').layers
generator, critic = keras.models.load_model('../input/wganmodels/v1_epoch_2600.h5').layers

In [ ]:
wgan = keras.models.Sequential([generator, critic])

# Generated Data

In [ ]:
#generated data
n_instances = X_train_concat.shape[0]
noise = tf.random.normal([n_instances, latent_space])
generated_image = generator(noise, training=False)
generated_image = generated_image.numpy()
generated_image = generated_image.reshape(n_instances*ntimes*1, n_features)
generated_image = scaler.inverse_transform(generated_image)

print(generated_image.shape)

fig, ax = plt.subplots(2,4, figsize=[20,10])

for i, group in enumerate(groups):
    ax.flatten()[i].plot(generated_image[:,0],generated_image[:,i+1],'o',c='orange')
    ax.flatten()[i].set_title(group)
    ax.flatten()[0].set_xlabel("time step")
    ax.flatten()[0].set_ylabel("concentration (ppm)")
    
plt.savefig('./output/Figures/Result/result_v'+str(k)+'.png')
print('save generated image')

X_train_concat_flatten2 = X_train_concat.reshape(X_train_concat.shape[0]*ntimes, n_features).astype('float32')
X_train_concat_flatten2 = scaler.inverse_transform(X_train_concat_flatten2)


In [ ]:
fig, ax = plt.subplots(2,4, figsize=[20,10])
for i, group in enumerate(groups):
    ax.flatten()[i].plot(X_train_concat_flatten2[:,0],X_train_concat_flatten2[:,i+1],'o')
    ax.flatten()[i].set_title(group)
    ax.flatten()[0].set_xlabel("time step")
    ax.flatten()[0].set_ylabel("concentration (ppm)")    
    
plt.savefig('./output/Figures/Result/sample_v'+str(k)+'.png')

# Predict with the GAN

In [ ]:
epochs = 2000 
n = 500
predict_points = 600 #X_train.shape[0]-n-ntimes

In [ ]:
mse = tf.keras.losses.MeanSquaredError()
optimizer = tf.keras.optimizers.Adam(1e-2) 

def mse_loss(inp, outp):
    inp = tf.reshape(inp, [-1, n_features]) 
    outp = tf.reshape(outp, [-1, n_features])
    print("input:",inp.shape)
    print("output:",outp.shape)
    
    return mse(inp, outp)



def optimize_coding(latent_values, real_coding, epochs):
    
    for epoch in range(epochs):
        opt_step(latent_values, real_coding)
        
    return latent_values  #returns the optimized input that generates the desired output


@tf.function
def opt_step(latent_values, real_coding):
    with tf.GradientTape() as tape:
        tape.watch(latent_values)
        gen_output = generator(latent_values, training=False)  #results from generator
        loss = mse_loss(real_coding, gen_output[:,:(ntimes - 1),:])   #codings_size is size of each output, ntimes is number of time levels. 
        #If training for 9 time levels, it finds the loss between the first 8 outputs from the generator and 8 real outputs

    gradient = tape.gradient(loss, latent_values)   #gradient of the loss ws to the input
    optimizer.apply_gradients(zip([gradient], [latent_values]))   #applies gradients to the input
    
    return loss


np.random.seed(0)
tf.random.set_seed(0)

real_coding = X_train_concat[n].reshape(1,-1)
real_coding = real_coding[:,:n_features*(ntimes - 1)]
real_coding = tf.constant(real_coding)
real_coding = tf.cast(real_coding, dtype=tf.float32)

latent_values = tf.random.normal([len(real_coding), latent_space])  
latent_values = tf.Variable(latent_values)     #make input a tensorflow variable so it can be trained
latent_values = optimize_coding(latent_values, real_coding, epochs)


X_predict = list(generator(latent_values).numpy().reshape(-1,codings_size))
gen_predict = X_predict[-1]
real_coding = np.concatenate((real_coding, gen_predict.reshape(1,-1)), axis=1)[:,n_features:]
real_coding = tf.constant(real_coding)
real_coding = tf.cast(real_coding, dtype=tf.float32)

for i in range(predict_points): #range(2000,len(X_train_concat)-1):
    latent_values = optimize_coding(latent_values, real_coding, epochs)
    gen_predict = generator(latent_values)[:,(ntimes - 1):,:].numpy()
    X_predict.append(gen_predict.flatten())
    real_coding = np.concatenate((real_coding, gen_predict.reshape(1,-1)), axis=1)[:,n_features:]
    real_coding = tf.constant(real_coding)
    real_coding = tf.cast(real_coding, dtype=tf.float32)
    
X_predict = np.array(X_predict)

In [ ]:
X_generated = scaler.inverse_transform(X_predict)
X_generated = X_generated.reshape(len(X_predict), n_features)
X_generated[X_generated<0] = 0 
np.save('X_generatedall.npy',X_generated)
        
    
fig, ax = plt.subplots(2,4, figsize=[20,10])
for i, group in enumerate(groups):
    #ax.flatten()[i].plot(X_generated[:,i+1], '-',c='orange', label='generated')
    ax.flatten()[i].plot(X_train[n:(n+ntimes+predict_points),i+1], '-', label='real')
    ax.flatten()[i].plot(X_generated[:,i+1], '-',c='orange', label='generated')
    ax.flatten()[i].set_title(group)
    ax.flatten()[i].legend()
    
plt.savefig('./output/Figures/Prediction/00entire_v'+str(k)+'_epoch_'+str(epochs)+'.png')